In [2]:
import pandas as pn

df_citibike = pn.read_excel("./ny_citibikes_raw.xlsx")

### 1. Dataset Exploration
  - What information does each column contain?
  - Are there missing or duplicated values?
  - What is the overall time span of the trips? TODO: ESTA MIERDA

Para comenzar a explorar los datos usamos los métodos: `sample()` e `info()`. El primero nos mostrará una muestra aleatoria de entradas en el dataset, el segundo una información general sobre tipos de datos, columnas, numero de entradas y valores nulos.

In [3]:
df_citibike.sample(5)

,Start Time,Stop Time,Start Station ID,Start Station Name,End Station ID,End Station Name,Bike ID,User Type,Birth Year,Age,Age Groups,Trip Duration,Trip_Duration_in_min,Month,Season,Temperature,Weekday
18976,2017-03-27 21:34:00,2017-03-27 21:38:00,3205,JC Medical Center,3192,Liberty Light Rail,26296,Subscriber,1990,31,25-34,266,4,3,Spring,16,Monday
15595,2017-03-13 08:27:00,2017-03-13 08:34:00,3203,Hamilton Park,3202,Newport PATH,26226,Subscriber,1978,43,35-44,420,7,3,Spring,17,Monday
7309,2017-02-14 20:20:00,2017-02-14 20:25:00,3183,Exchange Place,3187,Warren St,26199,Subscriber,1976,45,45-54,278,5,2,Winter,15,Tuesday
17491,2017-03-23 20:18:00,2017-03-23 20:20:00,3278,Monmouth and 6th,3272,Jersey & 3rd,26202,Subscriber,1988,33,25-34,145,2,3,Spring,16,Thursday
15566,2017-03-13 07:59:00,2017-03-13 08:03:00,3194,McGinley Square,3195,Sip Ave,24576,Subscriber,1977,44,35-44,215,4,3,Spring,17,Monday


In [4]:
df_citibike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20400 entries, 0 to 20399
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Start Time            20400 non-null  datetime64[ns]
 1   Stop Time             20400 non-null  datetime64[ns]
 2   Start Station ID      20400 non-null  int64         
 3   Start Station Name    20400 non-null  object        
 4   End Station ID        20400 non-null  int64         
 5   End Station Name      20399 non-null  object        
 6   Bike ID               20400 non-null  int64         
 7   User Type             20400 non-null  object        
 8   Birth Year            20400 non-null  int64         
 9   Age                   20400 non-null  int64         
 10  Age Groups            20400 non-null  object        
 11  Trip Duration         20400 non-null  int64         
 12  Trip_Duration_in_min  20400 non-null  int64         
 13  Month           

Hemos detectado que hay un valor nulo en la columna "End Station Name". Para asegurarnos haremos la suma de todos los valores nulos por columnas, por si se nos ha escapado alguo al observar el output de `info()` a ojo.

In [5]:
df_citibike.isna().sum()

Start Time              0
Stop Time               0
Start Station ID        0
Start Station Name      0
End Station ID          0
End Station Name        1
Bike ID                 0
User Type               0
Birth Year              0
Age                     0
Age Groups              0
Trip Duration           0
Trip_Duration_in_min    0
Month                   0
Season                  0
Temperature             0
Weekday                 0
dtype: int64

Confirmamos que unicamente hay una columna con un valor *not available*. Mostramos la entrada donde falta ese valor.

In [6]:
df_citibike[df_citibike['End Station Name'].isna()]

,Start Time,Stop Time,Start Station ID,Start Station Name,End Station ID,End Station Name,Bike ID,User Type,Birth Year,Age,Age Groups,Trip Duration,Trip_Duration_in_min,Month,Season,Temperature,Weekday
9858,2017-02-25 12:36:00,2017-02-25 12:44:00,3220,5 Corners Library,3211,NaN,24522,Subscriber,1963,58,55-64,532,9,2,Winter,11,Saturday


Por ser un único valor nulo en un conjunto de 20.000 entredas lo desecharemos.

In [7]:
df_citibike.dropna(inplace=True)

A continuación exploraremos si existen duplicados y cuantos. Al ejecutar la siguiente casilla de código veremos que existen 3555 entradas duplicadas. Los eliminaremos por ser insustanciales, quedándonos con unicamente 16845 entradas.

In [8]:
df_citibike.duplicated().sum()

np.int64(3555)

In [9]:
df_citibike.drop_duplicates(inplace=True)

In [10]:
df_citibike.shape

(16844, 17)

# 2. Basic Statistics
  - What is the average trip duration (in minutes)?
  - What is the minimum and maximum duration?
  - What are the most common start and end stations?

In [11]:
df_citibike.loc[:,"Trip_Duration_in_min"].mean()

np.float64(9.578306815483257)

In [12]:
df_citibike.loc[:,"Trip_Duration_in_min"].max()

np.int64(6515)

In [13]:
df_citibike.loc[:,"Trip_Duration_in_min"].min()

np.int64(1)

In [14]:
df_citibike.value_counts("Start Station ID").head(1)

Start Station ID
3186    2115
Name: count, dtype: int64

In [15]:
df_citibike.value_counts("End Station ID").head(1)

End Station ID
3186    2743
Name: count, dtype: int64

In [16]:
df_citibike.loc[df_citibike["End Station ID"]==3186]["End Station Name"].drop_duplicates()

26    Grove St PATH
Name: End Station Name, dtype: object

# 3. Users and Demographics
  - How many unique bikes were used?
  - What are the proportions of user types (Subscriber vs Customer)?
  - What is the age distribution of the users? Which age group uses the service the most?

In [17]:
df_citibike.loc[:, "Bike ID"].unique().size

500